In [1]:
import pandas as pd

from serlo.mysql import db

df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df.head()

/home/kulla/.local/share/asdf/installs/python/3.10.1/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,id,username,description,date,edits,reviews,comments,taxonomy
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",2014-03-01 20:36:21,163.0,172.0,78.0,784.0
1,2,devuser,NULL,2014-03-01 20:36:21,2.0,1.0,4.0,3.0
2,4,Deleted,NULL,2014-03-01 20:36:21,161.0,187.0,7.0,17.0
3,6,Legacy,NULL,2014-03-01 20:36:21,6986.0,6988.0,0.0,1908.0
4,9,David,NULL,2014-03-01 20:36:32,0.0,0.0,1.0,0.0


In [2]:
def has_link(description):
    if description == None or description == "NULL":
        return False
    
    return "http://" in description or "https://" in description

df["has_link"] = df["description"].map(has_link)
df["date"] = df["date"].map(pd.to_datetime)
df.head()

,id,username,description,date,edits,reviews,comments,taxonomy,has_link
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",2014-03-01 20:36:21,163.0,172.0,78.0,784.0,True
1,2,devuser,NULL,2014-03-01 20:36:21,2.0,1.0,4.0,3.0,False
2,4,Deleted,NULL,2014-03-01 20:36:21,161.0,187.0,7.0,17.0,False
3,6,Legacy,NULL,2014-03-01 20:36:21,6986.0,6988.0,0.0,1908.0,False
4,9,David,NULL,2014-03-01 20:36:32,0.0,0.0,1.0,0.0,False


In [29]:
df_delete = df[
    (df["edits"] + df["comments"] + df["reviews"] + df["taxonomy"] == 0) &
    df["has_link"] &
    (df["date"] < pd.to_datetime("2021-12-01"))
].copy()
df_delete.sort_values(["date"], inplace=True)
df_delete

,id,username,description,date,edits,reviews,comments,taxonomy,has_link
11495,145048,bestweightedblanket,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-09-28 15:01:22,0.0,0.0,0.0,0.0,True
11501,145056,Aida01,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-09-30 07:59:19,0.0,0.0,0.0,0.0,True
11521,145336,alldaypill122,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-10-03 11:28:41,0.0,0.0,0.0,0.0,True
11526,145351,fitnessdiet,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-10-04 12:05:08,0.0,0.0,0.0,0.0,True
11528,145377,nicholassmith,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-10-04 18:38:44,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...
22845,228130,ztorage,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-10-27 07:13:35,0.0,0.0,0.0,0.0,True
22848,228173,jasakontraktor,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-10-27 15:12:16,0.0,0.0,0.0,0.0,True
22851,228209,ruoumoyentuvn,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-10-27 18:26:14,0.0,0.0,0.0,0.0,True
22887,228454,dubaidesertsafarigroup,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-11-01 12:27:49,0.0,0.0,0.0,0.0,True


In [4]:
df_delete.to_csv("/tmp/bots-with-no-edits-to-delete-list.csv")

In [28]:
from serlo import SerloBot, Environment

bot = SerloBot(Environment.PRODUCTION)
bot.login()

Serlo-Username········
Serlo-Password········


In [30]:
import itertools

user_with_errors = []
total = len(df_delete)

for row, count in zip(df_delete.iterrows(), itertools.count(1)):
    user = row[1]
    user_id = user["id"]
    
    try:
        bot.api_call(query="""
            mutation {
                user {
                    deleteBots(input: { botIds: [%s]}) {
                        success
                    }
                }
            }
        """ % user_id)
        print(f"INFO: User with id {user_id} deleted... ({count}/{total})")
    except Exception as e:
        print(e)
        print(f"ERROR: User with id {user_id} could not be deleted...")
        user_with_errors.append(user)

INFO: User with id 145048 deleted... (1/6169)
INFO: User with id 145056 deleted... (2/6169)
INFO: User with id 145336 deleted... (3/6169)
INFO: User with id 145351 deleted... (4/6169)
INFO: User with id 145377 deleted... (5/6169)
INFO: User with id 145380 deleted... (6/6169)
INFO: User with id 145394 deleted... (7/6169)
INFO: User with id 145401 deleted... (8/6169)
INFO: User with id 145411 deleted... (9/6169)
INFO: User with id 145453 deleted... (10/6169)
INFO: User with id 145575 deleted... (11/6169)
INFO: User with id 145627 deleted... (12/6169)
INFO: User with id 145628 deleted... (13/6169)
INFO: User with id 145682 deleted... (14/6169)
INFO: User with id 145715 deleted... (15/6169)
INFO: User with id 145789 deleted... (16/6169)
INFO: User with id 145790 deleted... (17/6169)
INFO: User with id 145792 deleted... (18/6169)
INFO: User with id 145808 deleted... (19/6169)
INFO: User with id 145809 deleted... (20/6169)
INFO: User with id 145861 deleted... (21/6169)
INFO: User with id 145

INFO: User with id 149096 deleted... (173/6169)
INFO: User with id 149140 deleted... (174/6169)
INFO: User with id 149142 deleted... (175/6169)
INFO: User with id 149146 deleted... (176/6169)
INFO: User with id 149161 deleted... (177/6169)
INFO: User with id 149167 deleted... (178/6169)
INFO: User with id 149186 deleted... (179/6169)
INFO: User with id 149187 deleted... (180/6169)
INFO: User with id 149188 deleted... (181/6169)
INFO: User with id 149189 deleted... (182/6169)
INFO: User with id 149288 deleted... (183/6169)
INFO: User with id 149291 deleted... (184/6169)
INFO: User with id 149462 deleted... (185/6169)
INFO: User with id 149484 deleted... (186/6169)
INFO: User with id 149498 deleted... (187/6169)
INFO: User with id 149675 deleted... (188/6169)
INFO: User with id 149713 deleted... (189/6169)
INFO: User with id 149715 deleted... (190/6169)
INFO: User with id 149753 deleted... (191/6169)
INFO: User with id 149757 deleted... (192/6169)
INFO: User with id 149758 deleted... (19

INFO: User with id 156054 deleted... (344/6169)
INFO: User with id 156056 deleted... (345/6169)
INFO: User with id 156059 deleted... (346/6169)
INFO: User with id 156342 deleted... (347/6169)
INFO: User with id 156412 deleted... (348/6169)
INFO: User with id 156533 deleted... (349/6169)
INFO: User with id 156535 deleted... (350/6169)
INFO: User with id 156536 deleted... (351/6169)
INFO: User with id 156562 deleted... (352/6169)
INFO: User with id 156663 deleted... (353/6169)
INFO: User with id 157032 deleted... (354/6169)
INFO: User with id 157205 deleted... (355/6169)
INFO: User with id 157328 deleted... (356/6169)
INFO: User with id 157541 deleted... (357/6169)
INFO: User with id 157652 deleted... (358/6169)
INFO: User with id 157735 deleted... (359/6169)
INFO: User with id 157933 deleted... (360/6169)
INFO: User with id 157936 deleted... (361/6169)
INFO: User with id 158422 deleted... (362/6169)
INFO: User with id 158429 deleted... (363/6169)
INFO: User with id 158488 deleted... (36

INFO: User with id 168070 deleted... (515/6169)
INFO: User with id 168073 deleted... (516/6169)
INFO: User with id 168108 deleted... (517/6169)
INFO: User with id 168117 deleted... (518/6169)
INFO: User with id 168119 deleted... (519/6169)
INFO: User with id 168124 deleted... (520/6169)
INFO: User with id 168125 deleted... (521/6169)
INFO: User with id 168147 deleted... (522/6169)
INFO: User with id 168151 deleted... (523/6169)
INFO: User with id 168152 deleted... (524/6169)
INFO: User with id 168160 deleted... (525/6169)
INFO: User with id 168169 deleted... (526/6169)
INFO: User with id 168213 deleted... (527/6169)
INFO: User with id 168233 deleted... (528/6169)
INFO: User with id 168307 deleted... (529/6169)
INFO: User with id 168308 deleted... (530/6169)
INFO: User with id 168309 deleted... (531/6169)
INFO: User with id 168310 deleted... (532/6169)
INFO: User with id 168313 deleted... (533/6169)
INFO: User with id 168322 deleted... (534/6169)
INFO: User with id 168331 deleted... (53

INFO: User with id 171063 deleted... (686/6169)
INFO: User with id 171071 deleted... (687/6169)
INFO: User with id 171072 deleted... (688/6169)
INFO: User with id 171094 deleted... (689/6169)
INFO: User with id 171111 deleted... (690/6169)
INFO: User with id 171112 deleted... (691/6169)
INFO: User with id 171116 deleted... (692/6169)
INFO: User with id 171118 deleted... (693/6169)
INFO: User with id 171119 deleted... (694/6169)
INFO: User with id 171122 deleted... (695/6169)
INFO: User with id 171124 deleted... (696/6169)
INFO: User with id 171126 deleted... (697/6169)
INFO: User with id 171127 deleted... (698/6169)
INFO: User with id 171129 deleted... (699/6169)
INFO: User with id 171157 deleted... (700/6169)
INFO: User with id 171161 deleted... (701/6169)
INFO: User with id 171162 deleted... (702/6169)
INFO: User with id 171164 deleted... (703/6169)
INFO: User with id 171310 deleted... (704/6169)
INFO: User with id 171329 deleted... (705/6169)
INFO: User with id 171333 deleted... (70

...

Context creation failed: Token expired or invalid
ERROR: User with id 224312 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224329 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224330 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224332 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224338 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224340 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224344 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224359 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224360 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224374 could not be d

Context creation failed: Token expired or invalid
ERROR: User with id 224933 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224945 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224946 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224947 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224964 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224968 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224973 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224975 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224976 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 224977 could not be d

Context creation failed: Token expired or invalid
ERROR: User with id 225892 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226025 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226026 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226030 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226036 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226040 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226053 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226055 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226060 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 226065 could not be d

Context creation failed: Token expired or invalid
ERROR: User with id 227013 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227040 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227041 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227042 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227044 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227084 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227085 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227105 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227144 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227176 could not be d

Context creation failed: Token expired or invalid
ERROR: User with id 227922 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227935 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 227980 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 228002 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 228028 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 228029 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 228037 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 228038 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 228051 could not be deleted...
Context creation failed: Token expired or invalid
ERROR: User with id 228070 could not be d

In [33]:
bot = SerloBot(Environment.PRODUCTION)
bot.login()

user_with_errors2 = []
total = len(df_delete)

for user, count in zip(user_with_errors, itertools.count(1)):
    user_id = user["id"]
    
    try:
        bot.api_call(query="""
            mutation {
                user {
                    deleteBots(input: { botIds: [%s]}) {
                        success
                    }
                }
            }
        """ % user_id)
        print(f"INFO: User with id {user_id} deleted... ({count}/{total})")
    except Exception as e:
        print(e)
        print(f"ERROR: User with id {user_id} could not be deleted...")
        user_with_errors.append(user)

Serlo-Username········
Serlo-Password········
INFO: User with id 147475 deleted... (1/6169)
INFO: User with id 149067 deleted... (2/6169)
INFO: User with id 212279 deleted... (3/6169)
INFO: User with id 212280 deleted... (4/6169)
INFO: User with id 212281 deleted... (5/6169)
INFO: User with id 212282 deleted... (6/6169)
INFO: User with id 212286 deleted... (7/6169)
INFO: User with id 212287 deleted... (8/6169)
INFO: User with id 212288 deleted... (9/6169)
INFO: User with id 212309 deleted... (10/6169)
INFO: User with id 212343 deleted... (11/6169)
INFO: User with id 212344 deleted... (12/6169)
INFO: User with id 212346 deleted... (13/6169)
INFO: User with id 212347 deleted... (14/6169)
INFO: User with id 212348 deleted... (15/6169)
INFO: User with id 212349 deleted... (16/6169)
INFO: User with id 212351 deleted... (17/6169)
INFO: User with id 212353 deleted... (18/6169)
INFO: User with id 212354 deleted... (19/6169)
INFO: User with id 212355 deleted... (20/6169)
INFO: User with id 2123

INFO: User with id 213230 deleted... (173/6169)
INFO: User with id 213233 deleted... (174/6169)
INFO: User with id 213235 deleted... (175/6169)
INFO: User with id 213236 deleted... (176/6169)
INFO: User with id 213237 deleted... (177/6169)
INFO: User with id 213238 deleted... (178/6169)
INFO: User with id 213240 deleted... (179/6169)
INFO: User with id 213252 deleted... (180/6169)
INFO: User with id 213255 deleted... (181/6169)
INFO: User with id 213267 deleted... (182/6169)
INFO: User with id 213268 deleted... (183/6169)
INFO: User with id 213269 deleted... (184/6169)
INFO: User with id 213270 deleted... (185/6169)
INFO: User with id 213271 deleted... (186/6169)
INFO: User with id 213272 deleted... (187/6169)
INFO: User with id 213274 deleted... (188/6169)
INFO: User with id 213275 deleted... (189/6169)
INFO: User with id 213276 deleted... (190/6169)
INFO: User with id 213277 deleted... (191/6169)
INFO: User with id 213278 deleted... (192/6169)
INFO: User with id 213279 deleted... (19

INFO: User with id 215049 deleted... (344/6169)
INFO: User with id 215051 deleted... (345/6169)
INFO: User with id 215053 deleted... (346/6169)
INFO: User with id 215055 deleted... (347/6169)
INFO: User with id 215060 deleted... (348/6169)
INFO: User with id 215061 deleted... (349/6169)
INFO: User with id 215067 deleted... (350/6169)
INFO: User with id 215069 deleted... (351/6169)
INFO: User with id 215071 deleted... (352/6169)
INFO: User with id 215073 deleted... (353/6169)
INFO: User with id 215078 deleted... (354/6169)
INFO: User with id 215107 deleted... (355/6169)
INFO: User with id 215110 deleted... (356/6169)
INFO: User with id 215112 deleted... (357/6169)
INFO: User with id 215113 deleted... (358/6169)
INFO: User with id 215114 deleted... (359/6169)
INFO: User with id 215115 deleted... (360/6169)
INFO: User with id 215117 deleted... (361/6169)
INFO: User with id 215125 deleted... (362/6169)
INFO: User with id 215126 deleted... (363/6169)
INFO: User with id 215128 deleted... (36

INFO: User with id 215862 deleted... (515/6169)
INFO: User with id 215864 deleted... (516/6169)
INFO: User with id 215865 deleted... (517/6169)
INFO: User with id 215866 deleted... (518/6169)
INFO: User with id 215867 deleted... (519/6169)
INFO: User with id 215868 deleted... (520/6169)
INFO: User with id 215869 deleted... (521/6169)
INFO: User with id 215870 deleted... (522/6169)
INFO: User with id 215874 deleted... (523/6169)
INFO: User with id 215876 deleted... (524/6169)
INFO: User with id 215881 deleted... (525/6169)
INFO: User with id 215895 deleted... (526/6169)
INFO: User with id 215896 deleted... (527/6169)
INFO: User with id 215897 deleted... (528/6169)
INFO: User with id 215900 deleted... (529/6169)
INFO: User with id 215902 deleted... (530/6169)
INFO: User with id 215904 deleted... (531/6169)
INFO: User with id 215906 deleted... (532/6169)
INFO: User with id 215907 deleted... (533/6169)
INFO: User with id 215926 deleted... (534/6169)
INFO: User with id 215927 deleted... (53

INFO: User with id 218632 deleted... (686/6169)
INFO: User with id 218641 deleted... (687/6169)
INFO: User with id 218817 deleted... (688/6169)
INFO: User with id 218849 deleted... (689/6169)
INFO: User with id 218851 deleted... (690/6169)
INFO: User with id 219012 deleted... (691/6169)
INFO: User with id 219013 deleted... (692/6169)
INFO: User with id 219014 deleted... (693/6169)
INFO: User with id 219016 deleted... (694/6169)
INFO: User with id 219019 deleted... (695/6169)
INFO: User with id 219021 deleted... (696/6169)
INFO: User with id 219022 deleted... (697/6169)
INFO: User with id 219023 deleted... (698/6169)
INFO: User with id 219025 deleted... (699/6169)
INFO: User with id 219031 deleted... (700/6169)
INFO: User with id 219086 deleted... (701/6169)
INFO: User with id 219089 deleted... (702/6169)
INFO: User with id 219093 deleted... (703/6169)
INFO: User with id 219105 deleted... (704/6169)
INFO: User with id 219107 deleted... (705/6169)
INFO: User with id 219114 deleted... (70

...

not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User wit

not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User wit

not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User wit

not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User with id 213063 could not be deleted...
not all bots are users
ERROR: User with id 147475 could not be deleted...
not all bots are users
ERROR: User with id 212773 could not be deleted...
not all bots are users
ERROR: User wit

KeyboardInterrupt: 